# Part 2-Deploying a dashboard 

### Using dash to deploy a dashboard on the cleaned data  

Link to dataset:- https://www.kaggle.com/datasets/deepcontractor/australian-fatal-car-accident-data-19892021

In [1]:
#Importing all the necessary libraries
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output


C:\Users\yashp\AppData\Local\Temp\ipykernel_25064\214245429.py:8: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
C:\Users\yashp\AppData\Local\Temp\ipykernel_25064\214245429.py:9: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


### Loading the data 

In [2]:
#read the cleaned csv file 
df = pd.read_csv('cleanedCrash.csv')

In [3]:
#displaying the first 10 data values
df.head(10)

,Crash ID,State,Month,Year,Dayweek,Time,Crash Type,Bus Involvement,Heavy Rigid Truck Involvement,Articulated Truck Involvement,Speed Limit,Road User,Gender,Age,Christmas Period,Easter Period,Age Group,Day of week,Time of day
0,20212133,Vic,9,2021,Sunday,0:30,Single,No,No,No,0,Motorcycle rider,Male,38,No,No,26_to_39,Weekend,Night
1,20214022,SA,9,2021,Saturday,23:31,Multiple,No,No,No,110,Pedestrian,Female,28,No,No,26_to_39,Weekend,Night
2,20212096,Vic,9,2021,Saturday,23:00,Single,No,No,No,0,Passenger,Male,19,No,No,17_to_25,Weekend,Night
3,20212145,Vic,9,2021,Saturday,22:25,Single,No,No,No,0,Driver,Male,23,No,No,17_to_25,Weekend,Night
4,20212075,Vic,9,2021,Saturday,5:15,Single,No,No,No,0,Motorcycle rider,Male,46,No,No,40_to_64,Weekend,Night
5,20213034,Qld,9,2021,Saturday,4:00,Multiple,No,No,No,100,Motorcycle rider,Male,19,No,No,17_to_25,Weekend,Night
6,20215071,WA,9,2021,Saturday,19:10,Single,No,No,No,100,Driver,Female,20,No,No,17_to_25,Weekend,Night
7,20212015,Vic,9,2021,Thursday,19:33,Multiple,No,No,No,0,Other/-9,Male,17,No,No,17_to_25,Weekday,Night
8,20213026,Qld,9,2021,Wednesday,23:00,Multiple,No,No,No,0,Passenger,Female,2,No,No,0_to_16,Weekday,Night
9,20213092,Qld,9,2021,Saturday,2:00,Single,No,No,No,60,Driver,Male,47,No,No,40_to_64,Weekend,Night


## Creating the Dashboard 

In [4]:
# Create charts
accidents_by_month = px.line(df.groupby("Month").size().reset_index(name="Total Accidents"), x="Month", y="Total Accidents", title="Total Accidents by Month")

accidents_by_Year = px.line(df.groupby("Year").size().reset_index(name="Total Accidents"), x="Year", y="Total Accidents", title="Total Accidents by Year")

Crash_Type = px.pie(df, names="Crash Type", title="Crash Type")

public_holiday = px.pie(df, names="Easter Period", title="Crashes during Easter")

Busses = px.pie(df, names="Bus Involvement", title="Crashes involving Busses")

Trucks = px.pie(df, names="Heavy Rigid Truck Involvement", title="Crashes involving Heavy Trucks")

public_holiday_2 = px.pie(df, names="Christmas Period", title="Crashes during Christmas")

Trucks2=px.pie(df, names="Articulated Truck Involvement", title="Crashes involving Trucks")

Gender=px.pie(df, names="Gender", title="Crashes distributed by gender")

Time_of_day=px.pie(df, names="Time of day", title="Crashes distributed by Time of day")

State = px.histogram(df, x="State", color="Crash Type", title="Crash per state")

Speed_Limit = px.histogram(df, x="Speed Limit", color="Crash Type", title="Speed limit of the crashes")

#filtering database
df1=df[df["Time of day"]=='Night']

#graph for age groups only for crashes during night time 
Age_Group_nt =px.bar(df1.groupby("Age Group").size().reset_index(name="Count"), x="Age Group", y="Count", title="Accidents During the night by age groups")

day_of_week = px.bar(df.groupby("Dayweek").size().reset_index(name="Count"), x="Dayweek", y="Count", title="Accidents by Day of Week")

Road_User = px.bar(df.groupby("Road User").size().reset_index(name="Count"), x="Road User", y="Count", title="Accidents by types of Road User")

Age_Group =px.bar(df.groupby("Age Group").size().reset_index(name="Count"), x="Age Group", y="Count", title="Accidents by types of Age Group")

df_pivot = pd.pivot_table(df, values='Crash Type', index=df['Month'], columns=df['Dayweek'], aggfunc='count')

# Create app layout
app = dash.Dash(__name__)
app.layout = html.Div(children=[
    dcc.RangeSlider(
        id='year-slider',
        min=df['Year'].min(),
        max=df['Year'].max(),
        value=[1989, 2021],
        marks={str(year): str(year) for year in df['Year'].unique()},
        step=None
    ),
    #A slider for the year was put in place above 
    html.Br(),
    html.H1(children="Australia Road Accidents Dashboard"),
    dcc.Graph(id="line-chart", figure=accidents_by_month),
    dcc.Graph(id="line-chart2", figure=accidents_by_Year),
    dcc.Graph(id="pie-chart", figure=Crash_Type),
    dcc.Graph(id="pie-chart2", figure=public_holiday),
    dcc.Graph(id="pie-chart3", figure=Busses),
    dcc.Graph(id="pie-chart4", figure=public_holiday_2),
    dcc.Graph(id="pie-chart5", figure=Trucks),
    dcc.Graph(id="pie-chart6", figure=Trucks2),
    dcc.Graph(id="pie-chart7", figure=Gender),
    dcc.Graph(id="pie-chart8", figure=Time_of_day),
    dcc.Graph(id="histogram", figure=State),
    dcc.Graph(id="histogram2", figure=Speed_Limit),
    dcc.Graph(id="bar-chart", figure=day_of_week),
    dcc.Graph(id="bar-chart2", figure=Road_User),
    dcc.Graph(id="bar-chart3", figure=Age_Group),
    dcc.Graph(id="bar-chart4", figure=Age_Group_nt),
    dcc.Graph(id="pivot-table", figure=px.imshow(df_pivot, x=df_pivot.columns, y=df_pivot.index, title="Accidents by Month and Day of Week")),
    
])
# Define callback to update graphs
@app.callback(
    [Output("line-chart", "figure"), Output("line-chart2", "figure"), Output("pie-chart", "figure"), Output("pie-chart2", "figure"), Output("pie-chart3", "figure"), Output("pie-chart4","figure"), Output("pie-chart5", "figure"), Output("pie-chart6", "figure"), Output("pie-chart7", "figure"), Output("pie-chart8", "figure"), Output("histogram", "figure"), Output("histogram2", "figure"), Output("bar-chart", "figure"), Output("bar-chart2", "figure"), Output("bar-chart3", "figure"), Output("bar-chart4", "figure"), Output("pivot-table", "figure")],                   
    [Input("year-slider", "value")]
)
def update_graphs(year_range):
    filtered_df = df[(df["Year"] >= year_range[0]) & (df["Year"] <= year_range[1])]
    
    updated_accidents_by_Month = px.line(filtered_df.groupby("Month").size().reset_index(name="Total Accidents"), x="Month", y="Total Accidents", title="Total Accidents by Month")
    
    updated_accidents_by_Year = px.line(filtered_df.groupby("Year").size().reset_index(name="Total Accidents"), x="Year", y="Total Accidents", title="Total Accidents by Year")
    
    updated_Crash_Type = px.pie(filtered_df, names="Crash Type", title="Crash Type")
    
    updated_public_holiday = px.pie(filtered_df, names="Easter Period", title="Crashes during Easter")
    
    updated_Busses = px.pie(filtered_df, names="Bus Involvement", title="Crashes involving Busses")

    updated_public_holiday_2 = px.pie(filtered_df, names="Christmas Period", title="Crashes during Christmas")
    
    updated_Trucks = px.pie(filtered_df, names="Heavy Rigid Truck Involvement", title="Crashes involving Heavy Trucks")
    
    updated_Trucks2 =px.pie(filtered_df, names="Articulated Truck Involvement", title="Crashes involving Trucks")
    
    updated_Gender =px.pie(filtered_df, names="Gender", title="Crashes distributed by gender")
    
    updated_Time_of_day =px.pie(filtered_df, names="Time of day", title="Crashes distributed by Time of day")
    
    updated_State = px.histogram(filtered_df, x="State", color="Crash Type", title="Crash per state")
    
    updated_Speed_Limit = px.histogram(filtered_df, x="Speed Limit", color="Crash Type", title="Speed limit of the crashes")
    
    updated_day_of_week = px.bar(filtered_df.groupby("Dayweek").size().reset_index(name="Count"), x="Dayweek", y="Count", title="Accidents by Day of Week")
    
    updated_Road_User = px.bar(filtered_df.groupby("Road User").size().reset_index(name="Count"), x="Road User", y="Count", title="Accidents by types of Road User")
    
    updated_Age_Group = px.bar(filtered_df.groupby("Age Group").size().reset_index(name="Count"), x="Age Group", y="Count", title="Accidents by types of Age Group")
    
    df1=filtered_df[filtered_df["Time of day"]=='Night']

    updated_Age_Group_nt =px.bar(df1.groupby("Age Group").size().reset_index(name="Count"), x="Age Group", y="Count", title="Accidents During the night by age groups")
    
    df_pivot = pd.pivot_table(filtered_df, values='Crash Type', index=df['Month'], columns=df['Dayweek'], aggfunc='count')
    
    updated_accidents_by_month_day = px.imshow(df_pivot, x=df_pivot.columns, y=df_pivot.index,title="Accidents by Month and Day of Week")
    
    return updated_accidents_by_Month, updated_accidents_by_Year, updated_Crash_Type, updated_public_holiday, updated_Busses, updated_public_holiday_2, updated_Trucks, updated_Trucks2, updated_Gender, updated_Time_of_day, updated_State, updated_Speed_Limit, updated_day_of_week, updated_Road_User, updated_Age_Group, updated_Age_Group_nt, updated_accidents_by_month_day



## Calling the main block to run the app 

In [5]:
# Run app
if __name__ == "__main__":
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
